# Пример использования Pytorch в задаче классификации изображений

In [ ]:
%load_ext autoreload
%autoreload 2
%pylab inline

import os
from pathlib import Path
from collections import Counter

import cv2
import pandas as pd
import torch

## Подготовка данных

In [ ]:
## загрузите датасет https://drive.google.com/open?id=0BxYys69jI14kYVM3aVhKS1VhRUk 

pdata = Path('../../../UTKFace/')

In [ ]:
fns = list(pdata.glob('*.jpg'))
fns = [fn for fn in fns if len(str(fn).split('/')[-1].split('_'))==4 and '__' not in str(fn)]
print(len(fns))
fns[:3]

In [ ]:
def show_img(fn):
    plt.figure()
    img = cv2.imread(str(fn))[:,:,::-1]
    plt.imshow(img)
    
for _ in range(2): show_img(np.random.choice(fns))

In [ ]:
i2fn = fns
fn2i = {fn:i for i,fn in enumerate(i2fn)}
bs_fns = [fn.parts[-1] for fn in fns]
bs_fns[:3]

In [ ]:
i2age, i2gender, i2race = zip(*[bs_fn.split('_')[:3] for bs_fn in bs_fns])
i2age = np.array(i2age, dtype=np.float32)
i2gender = np.array(i2gender, dtype=np.int64)
i2race = np.array(i2race, dtype=np.int64)

o2gender = {0: 'male', 1: 'female'}
o2race = dict(list(enumerate(('White', 'Black', 'Asian', 'Indian', 'Others'))))

i2race_verbose = [o2race[int(o)] for o in i2race]
i2gender_verbose = [o2gender[int(o)] for o in i2gender]
print(Counter(i2gender_verbose))

In [ ]:
Counter(i2race_verbose).most_common()

In [ ]:
sorted(
    Counter(list(zip(i2race_verbose, i2gender_verbose))).items(),
    key=lambda x: x[1], reverse=True)

In [ ]:
def show_random_img():
    fn = np.random.choice(fns)
    i = fn2i[fn]
    show_img(fn)
    plt.title(f'age: {i2age[i]}; gender: {i2gender_verbose[i]}; race: {i2race_verbose[i]}')
    
for _ in range(5): show_random_img()

In [ ]:
df = pd.DataFrame({
    'img_name': bs_fns,
    'age': i2age,
    'gender': i2gender,
    'race': i2race})
df['is_train'] = np.random.choice(2, size=len(df), p=[0.2, 0.8])
df.head()

# Обучение моделей

## Классификация пола

In [ ]:
from collections import namedtuple
from pathlib import PosixPath
from tqdm import tqdm_notebook as tqdm

import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision.transforms import Compose, ToTensor, Normalize

from dataloaders import ImagesDataset
from model import ResnetModel
from train_utils import train, validate
from train_utils import AccuracyMetric, AccuracyPart

Предобученные модели и эмбединги вы можете скачать [отсюда](https://drive.google.com/open?id=1LcaIDe0AIWe_MzS2BBHxtGKUy0F8J73P).

In [ ]:
args = namedtuple('args', [])
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

args.device = device
args.print_freq = 1
args.batch_size = 128

In [ ]:
df = pd.read_csv('img2targets.csv')
df["img_name"] = df["img_name"].apply(lambda x: pdata / x)
df.head()

### Определение даталоадеров

Нам необходимо два загрузчика данных — для тренировки и для тестирования. 

dataset[i] должен возвращать преобразованную в тензор картинку и соответствующие лейблы (их может быть несколько)


In [ ]:
def get_data(classes, transforms):
    datasets = {
        x: ImagesDataset(
            df=df, image_paths_name="img_name", labels_names=classes, 
            is_train=x == "train", transform=transforms)
        for x in ["train", "dev"]}

    dataloaders = {
        x: DataLoader(
            dataset=datasets[x], batch_size=args.batch_size, 
            shuffle=True, num_workers=0, pin_memory=False, 
            drop_last=True) 
        for x in ["train", "dev"]}
    
    return datasets, dataloaders

In [ ]:
transforms = Compose([
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

datasets, dataloaders = get_data(classes=["gender"], transforms=transforms)

### Модель, лосс, оптимизатор

Для классификации мы будем использовать предобученный Resnet18. После feature extraction добавим два полносвязных слоя с дропаутом и релу.

In [ ]:
model = ResnetModel(2, dp=0.5).to(device)
model.head

In [ ]:
# в качестве лосса -- стандартная кросс энтропия
criterion = nn.modules.loss.CrossEntropyLoss()
optimizer = torch.optim.Adam(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=1e-3)

# = через три эпохи уменьшить learning rate в 10 раз
scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer=optimizer, step_size=3, gamma=0.1)

# метрика за которой хотим "следить" во время обучения
metrics = [AccuracyMetric()]

In [ ]:
def train_model(model, dataloaders, criterion, optimizer,
                scheduler, metrics, epochs=5):
    for epoch in tqdm(range(epochs)):
        scheduler.step()
        train(dataloaders["train"], model, criterion, optimizer, epoch, args, metrics=metrics)
        validate(dataloaders["dev"], model, criterion, args, metrics=metrics)

In [ ]:
## если у вас cpu, то рекомендуетс пропустить блок с обучением и загрузить обученную модель
model.load_state_dict(torch.load("gender_model.pth", map_location=device))
validate(dataloaders["dev"], model, criterion, args, metrics=metrics)

In [ ]:
# обучение лучше производить на gpu. для инференса на cpu воспользуйтесь загрузкой модели 
train_model(model, dataloaders, criterion, optimizer, scheduler, metrics, epochs=5)

In [ ]:
torch.save(model.state_dict(), "gender_model.pth")

## Классификация расы

In [ ]:
datasets, dataloaders = get_data(classes=["race"], transforms=transforms)

In [ ]:
# меняем только количество классов
model = ResnetModel(5, dp=0.5).to(device)
optimizer = torch.optim.Adam(
    filter(lambda p: p.requires_grad, model.parameters()), lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer=optimizer, step_size=3, gamma=0.1)

In [ ]:
## если у вас cpu, то рекомендуется пропустить блок с обучением и загрузить обученную модель
# model.load_state_dict(torch.load("race_model.pth", map_location=device))
# validate(dataloaders["dev"], model, criterion, args, metrics=metrics)

In [ ]:
train_model(model, dataloaders, criterion, optimizer, scheduler, metrics, epochs=5)

In [ ]:
torch.save(model.state_dict(), "race_model.pth")

## Multi-task 

In [ ]:
datasets, dataloaders = get_data(classes=["race", "gender"], transforms=transforms)

In [ ]:
# финальный лосс -- сумма лоссов по каждой из задач
def multi_task_loss(input, targets):
    race_loss = nn.functional.cross_entropy(input[:, :5], targets[:, 0])
    gender_loss = nn.functional.cross_entropy(input[:, 5:], targets[:, 1])
    return race_loss + gender_loss

metrics = [AccuracyPart(name="RaceAcc", output_slice=slice(0, 5), target_column=0),
           AccuracyPart(name="GenderAcc", output_slice=slice(5, 7), target_column=1)]

In [ ]:
# меняем количество классов и лосс
model = ResnetModel(7, dp=0.5).to(device)
criterion = multi_task_loss
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer=optimizer, step_size=3, gamma=0.1)

In [ ]:
## если у вас cpu, то рекомендуется пропустить блок с обучением и загрузить обученную модель
# model.load_state_dict(torch.load("race_and_gender_model.pth", map_location=device))
# validate(dataloaders["dev"], model, criterion, args, metrics=metrics)

In [ ]:
train_model(model, dataloaders, criterion, optimizer, scheduler, metrics, epochs=5)

In [ ]:
torch.save(model.state_dict(), "race_and_gender_model.pth")

In [ ]:
def try_multi_on_img(dataset, idx):
    tensor, label = dataset[idx]
    with torch.no_grad():
        model.eval()
        prediction = model(tensor.unsqueeze(0)).detach().cpu().numpy()[0]
        race = np.argmax(prediction[:5])
        gender = np.argmax(prediction[5:])
        show_img(dataset.image_paths[idx])
        plt.title(f"Real: race - {o2race[label[0]]},  gender - {o2gender[label[1]]}. \n \
        Predicted: race - {o2race[race]}, gender - {o2gender[gender]}")

In [ ]:
model = ResnetModel(7, dp=0.5).to(device)
state_dict = torch.load("race_and_gender_model.pth", map_location="cpu")
model.load_state_dict(state_dict)


for _ in range(4): try_multi_on_img(datasets["dev"], np.random.randint(len(datasets["dev"])))

# Сравнение эмбеддингов

Посмотрим, как выглядят признаки перед последним слоем для разных моделей. Размерность таких признаков — 512.

Для этого воспользуемся функцией get_embedding() нашего классификатора. 

In [ ]:
def compute_embeddings(model, dataloader):
    all_embeddings = []
    all_labels = []
    with torch.no_grad():
        model.eval()
        for i, (data, labels) in tqdm(enumerate(dataloader)):
            all_labels.append(labels.numpy())
            data = data.to(device)
            embeddings = model.get_embedding(data).detach().cpu().numpy()
            all_embeddings.append(embeddings)
        
    return all_embeddings, all_labels

def save_for_projector(embeddings, metadata, name):
    embeddings = pd.DataFrame(np.concatenate(embeddings))
    metadata = pd.DataFrame(np.concatenate(metadata), columns=["Race", "Gender"])
    embeddings.to_csv(f"embeddings_{name}.tsv", header=None, index=None, sep="\t")
    metadata.to_csv(f"metadata_{name}.tsv", index=None, sep="\t")

In [ ]:
model = ResnetModel(7, dp=0.5).to(device)
state_dict = torch.load("race_and_gender_model.pth", map_location=device)
model.load_state_dict(state_dict)

embeddings, metadata = compute_embeddings(model, dataloaders["dev"])
save_for_projector(embeddings, metadata, "race&gender")

model = ResnetModel(2, dp=0.5).to(device)
state_dict = torch.load("gender_model.pth", map_location=device)
model.load_state_dict(state_dict)

embeddings, metadata = compute_embeddings(model, dataloaders["dev"])
save_for_projector(embeddings, metadata, "gender_only")

### Загрузив полученные файлы в https://projector.tensorflow.org/ получаем ...

Для гендерной модели, отчетливо выделяется кластера — male, female

In [ ]:
from IPython.display import Image
from IPython.core.display import HTML 
W = 400

In [ ]:
Image(filename="images/gender_only_gender.PNG", width=W)

При этом расы расположены "в перемешку"

In [ ]:
Image(filename="images/gender_only_race.PNG", width=W)

Модель, отвечающая сразу за две задачи, выучила соответствующие признаки, в пространстве которых объекты близки как по полу так и по расе

In [ ]:
Image(filename="images/gender&race_gender.PNG", width=W)

In [ ]:
Image(filename="images/gender&race_race.PNG", width=W)